In [9]:
import tensorflow as tf
import scipy.linalg as la
from core.TensorflowState import TensorflowState
from system.SystemParametersGeneral import SystemParametersGeneral
from math_functions.c_to_r_mat import CtoRMat

%pylab inline
import random as rd
import time
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

Populating the interactive namespace from numpy and matplotlib


In [2]:
'''
qubit_pi
two_modes_cnot
'''

simulation_system = "qubit_pi"

In [3]:
if simulation_system == "qubit_pi":
    class SystemParameters(SystemParametersGeneral):
        
        def __init__(self):
            total_time = 5
            SystemParametersGeneral.__init__(self,total_time)
            self.init_states()
            
        def init_states(self):
            # concerned states
            self.states_concerned_list = [0,self.mode_state_num**2]
            self.init_vectors()
            
            # Initialize initial and target states in numpy vector
            Ut_c = la.expm((0-1j)*self.total_time*self.H0_c)
            Ct_c = Ut_c.copy()

            Ct_c[self.mode_state_num**2,0] = 1
            Ct_c[0,self.mode_state_num**2] = 1
            Ct_c[0,0] = 0
            Ct_c[self.mode_state_num**2,self.mode_state_num**2] = 0

            self.initial_state = self.identity
            self.target_state = CtoRMat(Ct_c)

In [4]:
with tf.device('/gpu:0'):
    tfs = TensorflowState(SystemParameters())
    graph = tfs.build_graph()

Building graph:
State initialized.
Operators initialized.
Operators weight initialized.
Intermediate states initialized.
Propagator initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!


In [10]:
class Analysis:
    
    def __init__(self, tf_final_state, tf_Hx, tf_Hz,tf_unitary_scale, tf_inter_vecs):
        self.sys_para = SystemParameters()
        self.tf_final_state = tf_final_state
        self.tf_Hx = tf_Hx
        self.tf_Hz = tf_Hz
        self.tf_unitary_scale = tf_unitary_scale
        self.tf_inter_vecs = tf_inter_vecs
    
    def RtoCMat(self,M):
        state_num = self.sys_para.state_num
        M_real = M[:state_num,:state_num]
        M_imag = M[state_num:2*state_num,:state_num]
        
        return (M_real+1j*M_imag)
        
    def get_final_state(self):
        M = self.tf_final_state.eval()
        CMat = self.RtoCMat(M)
        np.save("./data/GRAPE-M", np.array(CMat))
        return CMat
        
    def get_ops_weight(self):        
        Hx = self.tf_Hx.eval()
        Hz = self.tf_Hz.eval()
        #np.save("./data/GRAPE-control", np.array(ops_weight))
        return [Hx,Hz]
    
    def get_inter_vecs(self):
        state_num = self.sys_para.state_num
        inter_vecs_mag_squared = []
        for tf_inter_vec in self.tf_inter_vecs:
            inter_vec = tf_inter_vec.eval()
            inter_vec_real = 0.5*(inter_vec[0:state_num,:]+inter_vec[state_num:2*state_num,:])
            inter_vec_imag = 0.5*(inter_vec[state_num:2*state_num,:] - inter_vec[0:state_num,:])

            inter_vec_mag_squared = np.square(np.absolute(inter_vec_real+1j*inter_vec_imag))
            inter_vecs_mag_squared.append(inter_vec_mag_squared)
        np.save("./data/GRAPE-inter_vecs", np.array(inter_vecs_mag_squared))
        return inter_vecs_mag_squared

In [11]:
class Convergence:

    
    def reset_convergence(self):
        self.costs=[]
        self.reg_costs = []
        self.iterations=[]
        self.learning_rate=[]
        self.last_iter = 0
        self.accumulate_rate = 1.00

    def update_convergence(self,last_cost, last_reg_cost, anly):
        self.last_cost = last_cost
        self.last_reg_cost = last_reg_cost
          
        self.anly = anly
        self.plot_summary()
    
    def get_convergence(self):
        self.costs.append(self.last_cost)
        self.reg_costs.append(self.last_reg_cost)
        self.iterations.append(self.last_iter)
        self.last_iter+=self.update_step
        
    def plot_inter_vecs(self,pop_inter_vecs):
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[0,1:]),label='g00')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[1,1:]),label='g01')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[self.sys_para.mode_state_num,1:]),label='g10')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[self.sys_para.mode_state_num+1,1:]),label='g11')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
    ,np.array(pop_inter_vecs[self.sys_para.mode_state_num**2:2*self.sys_para.mode_state_num**2,1:].sum(axis=0))
             ,label='e(012)(012)')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
    ,np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2:3*self.sys_para.mode_state_num**2,1:].sum(axis=0))
             ,label='f(012)(012)') 
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
             ,np.array(pop_inter_vecs[3*self.sys_para.mode_state_num**2:4*self.sys_para.mode_state_num**2,1:].sum(axis=0)) +\
             np.array(pop_inter_vecs[2,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+2*self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+2*self.sys_para.mode_state_num,1:])
             ,label='forbidden')
        
        ylabel('Population')
        ylim(0,1)
        xlabel('Time (ns)')
        legend(loc=6)
        
    def plot_inter_vecs_v2(self,pop_inter_vecs):
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[0,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2,1:]),label='00')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[1,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+1,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+1,1:]),label='01')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+self.sys_para.mode_state_num,1:]),label='10')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[self.sys_para.mode_state_num+1,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+self.sys_para.mode_state_num+1,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+self.sys_para.mode_state_num+1,1:]),label='11')
         
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
             ,np.array(pop_inter_vecs[3*self.sys_para.mode_state_num**2:4*self.sys_para.mode_state_num**2,1:].sum(axis=0)) +\
             np.array(pop_inter_vecs[2,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+2*self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+2*self.sys_para.mode_state_num,1:])
             ,label='forbidden')
        
        ylabel('Population')
        ylim(0,1)
        xlabel('Time (ns)')
        legend(loc=6)
        
    def plot_inter_vecs_v3(self,pop_inter_vecs):
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[0,1:]),label='g00')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[self.sys_para.mode_state_num**2,1:]),label='e00')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(pop_inter_vecs[self.sys_para.mode_state_num,1:]),label='g10')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
             ,np.array(pop_inter_vecs[self.sys_para.mode_state_num+self.sys_para.mode_state_num**2,1:]),label='e10')
#        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
#    ,np.array(pop_inter_vecs[self.sys_para.mode_state_num**2:2*self.sys_para.mode_state_num**2,1:].sum(axis=0))
#             ,label='e(012)(012)')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
    ,np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2:3*self.sys_para.mode_state_num**2,1:].sum(axis=0))
             ,label='f(012)(012)') 
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)])
             ,np.array(pop_inter_vecs[3*self.sys_para.mode_state_num**2:4*self.sys_para.mode_state_num**2,1:].sum(axis=0)) +\
             np.array(pop_inter_vecs[2,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+2,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[self.sys_para.mode_state_num**2+2*self.sys_para.mode_state_num,1:])+\
             np.array(pop_inter_vecs[2*self.sys_para.mode_state_num**2+2*self.sys_para.mode_state_num,1:])
             ,label='forbidden')
        
        ylabel('Population')
        ylim(0,1)
        xlabel('Time (ns)')
        legend(loc=6)
        
    def plot_summary(self):
        
        if not self.last_iter == 0:
            self.runtime = time.time() - self.start_time
            self.estimated_runtime = float(self.runtime * (self.max_iterations-self.last_iter) / self.last_iter)/(60*60)
        else:
            self.start_time = time.time()
            self.runtime = 0
            self.estimated_runtime = 0

        
        self.get_convergence()
        
        gs = gridspec.GridSpec(3+len(self.sys_para.states_concerned_list), 2)
        
        ## cost
        subplot(gs[0, :],title='Error = %.9f; Unitary Metric: %.5f; Runtime: %.1fs; Estimated Remaining Runtime: %.1fh' % (self.last_cost,
                                                                                                   self.anly.tf_unitary_scale.eval(),
                                                                                                 
                                                                                                  self.runtime,
                                                                                                  self.estimated_runtime))
        plot(array(self.iterations),array(self.costs),'bx-',label='loss')
        plot(array(self.iterations),array(self.reg_costs),'go-',label='reg loss')
        ylabel('Error')
        xlabel('Iteration')
        yscale('log')
        legend()
        numpy.save("./data/GRAPE-costs", np.array(self.costs))
    
        ## unitary evolution
        M = self.anly.get_final_state()
        subplot(gs[1, 0],title="operator: real")
        imshow(M.real,interpolation='none')
        clim(-1,1)
        colorbar()
        subplot(gs[1, 1],title="operator: imaginary")
        imshow(M.imag,interpolation='none')
        clim(-1,1)
        colorbar()
        
        ## operators
        subplot(gs[2, :],title="Operators")
        ops_weight = self.anly.get_ops_weight()
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),np.array(self.sys_para.ops_max_amp[0]*ops_weight[0]),'r',label='x')
        plot(array([self.sys_para.dt* ii for ii in range(self.sys_para.steps)]),(self.sys_para.qm_g1/(2*np.pi))\
             *np.array(self.sys_para.ops_max_amp[1]*ops_weight[1]),'c',label='(g/2pi)z')
        title('Optimized pulse')
        ylabel('Amplitude')
        xlabel('Time (ns)')
        legend()
        
        ## state evolution
        inter_vecs = self.anly.get_inter_vecs()
        
        for ii in range(len(self.sys_para.states_concerned_list)):
            subplot(gs[3+ii, :],title="Evolution")

            pop_inter_vecs = inter_vecs[ii]
            self.plot_inter_vecs_v3(pop_inter_vecs)        
        
        display.display(gcf())
        display.clear_output(wait=True)
        
    def __init__(self):
        # paramters
        self.sys_para = SystemParameters()
        self.update_step = 100
        self.conv_target = 1e-8
        self.max_iterations = 5000
        
        self.learning_rate_decay = self.max_iterations/2
        
        self.reg_alpha_coeff = 0.01
        
        self.z_reg_alpha_coeff = 0.01
        
        self.dwdt_reg_alpha_coeff = 0.0001
        self.d2wdt2_reg_alpha_coeff = 0.001*0.0001
        
        self.inter_reg_alpha_coeff = 100.0
        
        self.reset_convergence()
        
        self.fig = plt.figure(figsize=(15,50))

In [15]:
def run_session(single_simulation = False):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        conv = Convergence()
        
        
        print "Initialized"
        iterations = 0
        
        while True:
            if (single_simulation == False):
                max_iterations = conv.max_iterations
            else:
                max_iterations = 0
            learning_rate = 0.01 * np.exp(-float(iterations)/conv.learning_rate_decay)
            
            feed_dict = {tfs.learning_rate : learning_rate, tfs.z_reg_alpha_coeff: conv.z_reg_alpha_coeff,
                        tfs.reg_alpha_coeff: conv.reg_alpha_coeff, 
                         tfs.dwdt_reg_alpha_coeff: conv.dwdt_reg_alpha_coeff,
                         tfs.d2wdt2_reg_alpha_coeff: conv.d2wdt2_reg_alpha_coeff,
                         tfs.inter_reg_alpha_coeff:conv.inter_reg_alpha_coeff}
            _, l,rl = session.run([tfs.optimizer, tfs.loss, tfs.reg_loss], feed_dict=feed_dict)
            if (iterations % conv.update_step == 0):    
                
                # Plot convergence
                anly = Analysis(tfs.final_state,tfs.Hx,tfs.Hz,tfs.unitary_scale,tfs.inter_vecs)
                conv.update_convergence(l,rl,anly)
                
                # Save the variables to disk.
                save_path = tfs.saver.save(session, "./tmp/grape.ckpt")
                if (iterations >= max_iterations): #(l<conv.conv_target) or (iterations>=conv.max_iterations):
                    anly.get_ops_weight()
                    break
                    
                
            iterations+=1

In [ ]:
try:
    run_session()
except KeyboardInterrupt:
    display.clear_output()